In [1]:
from lda import *
from gensim.utils import simple_preprocess
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

# LDA Model Analysis

In [2]:
LOAD_SAVED = False
PRINT_COHERENCE = True

## Preprocessing
We will create bigrams and train LDA models for each of the four datasets we have. We will use the validation dataset to test the impact of using LDA output as features for the model and the test dataset to evaluate cross-domain generalization.

### Implicit Hate

In [3]:
implicit_df_train = pd.read_csv("./../../data/implicithate_train.csv")
lda_implicit, vectors_implicit_train, coherence_implicit = lda(implicit_df_train, load_from="../../data/lda_implicit.model") if LOAD_SAVED else lda(implicit_df_train)

In [4]:
if PRINT_COHERENCE:
    print('Coherence Score:', coherence_implicit)

Coherence Score: 0.3919226698945722


In [5]:
if not LOAD_SAVED:
    lda_implicit.save('../../saved-models/lda_implicit.model')

### COVID Hate

In [6]:
covid_df_train = pd.read_csv("./../../data/covidhate_train.csv")
lda_covid, vectors_covid_train, coherence_covid = lda(covid_df_train, load_from="../../saved-models/lda_covid.model") if LOAD_SAVED else lda(covid_df_train)

In [7]:
if PRINT_COHERENCE:
    print('Coherence Score:', coherence_covid)

Coherence Score: 0.4378289569222037


In [8]:
if not LOAD_SAVED:
    lda_covid.save('../../saved-models/lda_covid.model')

### Offensval

In [9]:
offenseval_df_train = pd.read_csv("./../../data/offenseval_train.csv")
lda_offenseval, vectors_offenseval_train, coherence_offenseval = lda(offenseval_df_train, load_from="../../saved-models/lda_offenseval.model") if LOAD_SAVED else lda(offenseval_df_train)

In [10]:
if PRINT_COHERENCE:
    print('Coherence Score:', coherence_offenseval)

Coherence Score: 0.33806384735719547


In [11]:
if not LOAD_SAVED:
    lda_offenseval.save('../../saved-models/lda_offenseval.model')

## Running SVMs with LDA features as input

### Create/Validate the SVMs

#### Implicit Hate

In [12]:
X = np.array(vectors_implicit_train)
y = np.array(implicit_df_train['label'])
svm_implicit = LinearSVC(tol=1e-3, class_weight="balanced").fit(X, y)

In [13]:
implicit_df_val = pd.read_csv("./../../data/implicithate_val.csv")
implicit_df_val = create_tokenized_column(implicit_df_val)
implicit_bigrams_val = create_bigrams(implicit_df_val["tokenized"])
implicit_dict_val, implicit_corpus_val = get_dict_and_corpus(implicit_bigrams_val)
implicit_val_vectors = get_vectors(lda_implicit, implicit_corpus_val)

In [14]:
preds = svm_implicit.predict(implicit_val_vectors)
print(classification_report(implicit_df_val['label'], preds, labels=[0, 1], target_names=['non-hate', 'hate']))

              precision    recall  f1-score   support

    non-hate       0.62      0.62      0.62      2150
        hate       0.36      0.37      0.36      1284

    accuracy                           0.52      3434
   macro avg       0.49      0.49      0.49      3434
weighted avg       0.52      0.52      0.52      3434



#### COVID Hate

In [15]:
X = np.array(vectors_covid_train)
y = np.array(covid_df_train['label'])
svm_covid = LinearSVC(tol=1e-3, class_weight="balanced").fit(X, y)

In [16]:
covid_df_val = pd.read_csv("./../../data/covidhate_val.csv")
covid_df_val = create_tokenized_column(covid_df_val)
covid_bigrams_val = create_bigrams(covid_df_val["tokenized"])
covid_dict_val, covid_corpus_val = get_dict_and_corpus(covid_bigrams_val)
covid_val_vectors = get_vectors(lda_covid, covid_corpus_val)

In [17]:
preds = svm_covid.predict(covid_val_vectors)
print(classification_report(covid_df_val['label'], preds, labels=[0, 1], target_names=['non-hate', 'hate']))

              precision    recall  f1-score   support

    non-hate       0.86      0.58      0.69       312
        hate       0.16      0.45      0.24        55

    accuracy                           0.56       367
   macro avg       0.51      0.52      0.46       367
weighted avg       0.75      0.56      0.62       367



#### Offenseval

In [18]:
X = np.array(vectors_offenseval_train)
y = np.array(offenseval_df_train['label'])
svm_offenseval = LinearSVC(tol=1e-3, class_weight="balanced").fit(X, y)

In [19]:
offenseval_df_val = pd.read_csv("./../../data/offenseval_val.csv")
offenseval_df_val = create_tokenized_column(offenseval_df_val)
offenseval_bigrams_val = create_bigrams(offenseval_df_val["tokenized"])
offenseval_dict_val, offenseval_corpus_val = get_dict_and_corpus(offenseval_bigrams_val)
offenseval_val_vectors = get_vectors(lda_offenseval, offenseval_corpus_val)

In [20]:
preds = svm_offenseval.predict(offenseval_val_vectors)
print(classification_report(offenseval_df_val['label'], preds, labels=[0, 1], target_names=['non-hate', 'hate']))

              precision    recall  f1-score   support

    non-hate       0.67      0.47      0.55      1733
        hate       0.36      0.57      0.44       915

    accuracy                           0.50      2648
   macro avg       0.52      0.52      0.50      2648
weighted avg       0.56      0.50      0.51      2648



### Running the SVMs using TF-IDF features

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(["from", "subject", "re", "edu", "use"])

#### Implicit Hate

In [22]:
implicit_tfidf = TfidfVectorizer(stop_words=stop_words, max_features=5000)
implicit_tfidf.fit(implicit_df_train['text'])
implicit_tfidf_train = implicit_tfidf.transform(implicit_df_train['text'])
implicit_tfidf_val = implicit_tfidf.transform(implicit_df_val['text'])
svm_implicit = LinearSVC(tol=1e-3, class_weight="balanced").fit(implicit_tfidf_train, implicit_df_train['label'])

In [23]:
preds = svm_implicit.predict(implicit_tfidf_val)
print(classification_report(implicit_df_val['label'], preds, labels=[0, 1], target_names=['non-hate', 'hate']))

              precision    recall  f1-score   support

    non-hate       0.77      0.73      0.75      2150
        hate       0.58      0.64      0.61      1284

    accuracy                           0.70      3434
   macro avg       0.68      0.68      0.68      3434
weighted avg       0.70      0.70      0.70      3434



#### COVID Hate

In [24]:
covid_tfidf = TfidfVectorizer(stop_words=stop_words, max_features=5000)
covid_tfidf.fit(covid_df_train['text'])
covid_tfidf_train = covid_tfidf.transform(covid_df_train['text'])
covid_tfidf_val = covid_tfidf.transform(covid_df_val['text'])
svm_covid = LinearSVC(tol=1e-3, class_weight="balanced").fit(covid_tfidf_train, covid_df_train['label'])

In [25]:
preds = svm_covid.predict(covid_tfidf_val)
print(classification_report(covid_df_val['label'], preds, labels=[0, 1], target_names=['non-hate', 'hate']))

              precision    recall  f1-score   support

    non-hate       0.93      0.93      0.93       312
        hate       0.60      0.62      0.61        55

    accuracy                           0.88       367
   macro avg       0.76      0.77      0.77       367
weighted avg       0.88      0.88      0.88       367



#### Offenseval

In [26]:
offenseval_tfidf = TfidfVectorizer(stop_words=stop_words, max_features=5000)
offenseval_tfidf.fit(offenseval_df_train['text'])
offenseval_tfidf_train = offenseval_tfidf.transform(offenseval_df_train['text'])
offenseval_tfidf_val = offenseval_tfidf.transform(offenseval_df_val['text'])
svm_offenseval = LinearSVC(tol=1e-3, class_weight="balanced").fit(offenseval_tfidf_train, offenseval_df_train['label'])

In [27]:
preds = svm_offenseval.predict(offenseval_tfidf_val)
print(classification_report(offenseval_df_val['label'], preds, labels=[0, 1], target_names=['non-hate', 'hate']))

              precision    recall  f1-score   support

    non-hate       0.79      0.77      0.78      1733
        hate       0.59      0.62      0.61       915

    accuracy                           0.72      2648
   macro avg       0.69      0.70      0.69      2648
weighted avg       0.72      0.72      0.72      2648

